In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd ../gdrive/MyDrive/hw1ml/

In [ ]:
%ls


develop on local environment

In [ ]:
# Install useful stuff
! apt install --yes ssh screen nano htop ranger git > /dev/null
# SSH setting
! echo "root:carbonara" | chpasswd
! echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
! echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! service ssh restart > /dev/null
# Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
# Run ngrok
authtoken = "20Apljvp3ViVnx3mzPGy8RN3VFt_4bzufXGF9Kkgm43PmRugS"
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
! sleep 3
# Get the address for SSH
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh
print(str_ssh)

use once you open the notebook

In [ ]:
%cd ../hw1ml

this suppress the warnings just to see the confusion matrix and the scores

In [ ]:
!pip install shutup

In [ ]:

##At the top of the code
import shutup;
shutup.please()

function to print metrics and confusion matrix given a model to fit for the classfication

In [ ]:
def classificationResults(x_train,x_test,y_train,y_test,models):
    if type(models) is not list:
        models=[models]
    y_train=y_train[:,0]
    y_test=y_test[:,0]

    for model in models:
        print(f"the model is {model}")
        model.fit(x_train,y_train)
        y_pred_train=model.predict(x_train)
        y_pred_test=model.predict(x_test)
        print("train report")
        print(classification_report(y_train, y_pred_train))
        print("test report")
        print(classification_report(y_test, y_pred_test))
        print("\nConfusion Matrix test:")

        cm = confusion_matrix(y_test, y_pred_test)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        print("\n\n\n")

function to print the metrics for the regression

In [ ]:
def regressionResults(x_train,x_test,y_train,y_test,models,sample_weight=None):
    if type(models) is not list:
        models=[models]
    y_train=y_train[:,1]
    y_test=y_test[:,1]
    for model in models:
        print(f"the model is {model}")
        model.fit(x_train,y_train)
        y_pred_train=model.predict(x_train)
        y_pred_test=model.predict(x_test)
        print("Train MSE: ",np.mean((y_train-y_pred_train)**2))
        print("Train MAE: ",np.mean(np.abs(y_train-y_pred_train)))
        print("Train R2: ",r2_score(y_train,y_pred_train))
        print("Test MSE: ",np.mean((y_test-y_pred_test)**2))
        print("Test MAE: ",np.mean(np.abs(y_test-y_pred_test)))
        print("Test R2: ",r2_score(y_test,y_pred_test))
        print("\n\n\n")
    

read the dataset and divide x and y

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import cosine_similarity
import random
import warnings

#make everything reproducible
random.seed(50)
np.random.seed(50)


dataset=pd.read_csv("train_set.tsv", sep='\t', header=0)
dataset_numpy=dataset.to_numpy()
x=dataset_numpy[:,:-2]
y=dataset_numpy[:,-2:]



those are 2 function that allows to see that solving with good performances just one of the two tasks would be sufficient to solve also the other task so use it just to see this think not for the true evaluation of the models

In [ ]:
#just for seeing the strange results described with the y values in the train set
#add cpa to x for the classification
x=np.hstack((x,y[:,1].reshape(-1,1)))

In [ ]:
#just for seeing the strange results described with the y values in the train set
#add collision to the regression
x=np.hstack((x,y[:,0].reshape(-1,1)))

add new features

In [ ]:
#if you want
#add the cosine of the angle from the north as a new feature 
# add the distance in x and y afrom the uavs to their target a
print(f"shape before {x.shape}")
for i in range(0, x.shape[1],7):
    distance_x=x[:,i+1]-x[:,i+5]
    distance_y=x[:,i+2]-x[:,i+6]
    cosine_north=np.cos(x[:,i])
    x=np.append(x, np.array([distance_x]).T, axis=1)
    x=np.append(x, np.array([distance_y]).T, axis=1)
    x=np.append(x, np.array([cosine_north]).T, axis=1)
print(f"shape after {x.shape}")  



use the minmax scaler

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(x)
x=scaler.transform(x)

use the standard scaler

In [ ]:
scaler = StandardScaler()
scaler.fit(x)
x=scaler.transform(x)


if you want to normalize the data

In [ ]:
from sklearn.preprocessing import normalize
x=normalize(x)

trying to use pca features

In [ ]:

from sklearn.decomposition import PCA
pca = PCA(n_components=4)
x = pca.fit_transform(x)

trying to use svd features

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=15)
x=svd.fit_transform(x)

trying to extract nonlinear features

In [ ]:
from sklearn.preprocessing import QuantileTransformer
x=QuantileTransformer().fit_transform(x)

split the dataset in train and test sets

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=45)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
# print(f"the classes and the number of occurrences in the train set are {np.unique(y_train, return_counts=True)}")
# print(f"the classes and the number of occurrences in the test set are {np.unique(y_test, return_counts=True)}")

In [ ]:
#import pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from sklearn.svm import SVC

#make a pipeline
pipe = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])

#grid search
param_grid = {
                'svc__kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                'scaler': [StandardScaler(), MinMaxScaler(), RobustScaler(), QuantileTransformer(), Normalizer(), MaxAbsScaler(), None],
            }
grid = GridSearchCV(pipe, param_grid, refit=True, verbose=1)
grid.fit(x_train, y_train[:,0])

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)
print(grid.score(x_test, y_test[:,0]))




find the closest point in cosine similarity and set the last row to the closest point in the train last row as described in the report to see if something could be done better

In [ ]:
#this can add as x the closest if you want to try the classification helping the model 
#find the closest point to each point in the train set and add it as a feature to the x test
l_test=[]
for i in range(x_test.shape[0]):
    index=0
    max_similar=0
    count=0
    for j in range(x_train.shape[0]):
        cossim=cosine_similarity(x_test[i].reshape(1,-1), x_train[j].reshape(1,-1))
        count+=cossim
        if cossim> max_similar:
            max_similar=cossim
            index=j
    l_test.append(y_train[index][1])
    # print(max_similar, count/x_train.shape[0])
l_test=np.array(l_test)
x_test=np.hstack((x_test,l_test.reshape(-1,1)))
print("finished adding the test features")
x_train=np.hstack((x_train,y_train))



In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)
model.fit(x_train,y_train[:,1])
l_test=model.predict(x_test)
x_test=np.hstack((x_test,l_test.reshape(-1,1)))
l_train=model.predict(x_train)
x_train=np.hstack((x_train,l_train.reshape(-1,1)))




compute class weights

In [ ]:
#this computest class weights for the classificaiton but its not useful 
#could be used to balance the outputs of the model to give less importance to the majority class
#and more importance to the minority class

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train[:,0]), 
    y=y_train[:,0]
    )
class_weight=dict(enumerate(class_weights))
sample_weight=np.array([class_weight[y_train[i,0]] for i in range(y_train.shape[0])])
class_weight

number of samples for each class

In [ ]:
#count the number of occurrences of each class
print(f"the classes and the number of occurrences in the train set are {np.unique(y_train[:,0], return_counts=True)}")

use smota with a mix of under and over sampling if you want to use this for the classification problem

In [ ]:
#you can change the parameters to see the different performances
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
#i let the last classas it is since they are reaaaaly low in n of samples
over_sampler = RandomOverSampler(sampling_strategy={}, random_state=0)
under_sampler = RandomUnderSampler(sampling_strategy={0:100,1:100}, random_state=0)
smote=SMOTE(sampling_strategy={2:100,3:100},random_state=100)


x_train,y_train=under_sampler.fit_resample(x_train,y_train[:,0])
x_train,y_train=over_sampler.fit_resample(x_train,y_train)
x_train,y_train=smote.fit_resample(x_train,y_train)

y_train=y_train.reshape(-1,1)
print(f"the classes and the number of occurrences in the train set are {np.unique(y_train, return_counts=True)}")

a list of models with their performances metrics

In [ ]:
#test all classification models
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.dummy import DummyClassifier
#import mlp classifier
from sklearn.neural_network import MLPClassifier

models=[
tree.DecisionTreeClassifier(),
RandomForestClassifier(),
AdaBoostClassifier(),
# GradientBoostingClassifier(),
LogisticRegression(),
SVC(),
KNeighborsClassifier(),
GaussianNB(),
BaggingClassifier(),
BalancedBaggingClassifier(),
BalancedRandomForestClassifier(),
# RUSBoostClassifier(),
# EasyEnsembleClassifier(),
# MLPClassifier(),
DummyClassifier( strategy='most_frequent')
]

classificationResults(x_train,x_test,y_train,y_test,models)


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline

over_sampler = RandomOverSampler(sampling_strategy={4:7})


imba_pipeline = make_pipeline(RandomOverSampler(sampling_strategy={4:7}),SMOTE(random_state=42), 
                              RandomForestClassifier(n_estimators=100, random_state=13))
cross_val_score(imba_pipeline, x_train, y_train[:,0], scoring='f1_macro', cv=5)

In [ ]:
from sklearn.dummy import DummyClassifier
model=DummyClassifier(strategy='most_frequent')
model.fit(x_train,y_train[:,0])
classificationResults(x_train,x_test,y_train,y_test,[model])

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import tree

imba_pipeline = make_pipeline(RandomOverSampler(sampling_strategy={4:7}),SMOTE(random_state=42), 
                            tree.DecisionTreeClassifier())
params={
    "max_depth": [3,5,10,15,20,None],
    "min_samples_split": [2,5,7,10],
    "min_samples_leaf": [1,2,5],
    "ccp_alpha": [0,0.00001,0.0001,0.0002,0.0005,0.001],
    }
new_params = {'decisiontreeclassifier__' + key: params[key] for key in params}
grid_imba = GridSearchCV(imba_pipeline, param_grid=new_params, cv=5, scoring='f1_macro',
                        return_train_score=True)
grid_imba.fit(x_train, y_train[:,0])
print(grid_imba.best_params_)
print(grid_imba.best_score_)

# model=tree.DecisionTreeClassifier(**clf.best_params_)
# classificationResults(x_train,x_test,y_train,y_test,[model])

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import tree

imba_pipeline = make_pipeline(RandomOverSampler(sampling_strategy={4:7}),SMOTE(random_state=42), 
                            SVC())
params={
    'C':[1, 10],
    'gamma':[1, 10],
    'kernel': ['rbf', 'poly', 'linear',"sigmoid"],
    'degree': [2, 3, 4,5],
    'class_weight': ['balanced', None],

    }
new_params = {'svc__' + key: params[key] for key in params}
grid_imba = GridSearchCV(imba_pipeline, param_grid=new_params, cv=5, scoring='f1_macro',
                        return_train_score=True)
grid_imba.fit(x_train, y_train[:,0])
print(grid_imba.best_params_)
print(grid_imba.best_score_)

# model=tree.DecisionTreeClassifier(**clf.best_params_)
# classificationResults(x_train,x_test,y_train,y_test,[model])

In [ ]:

from sklearn.model_selection import GridSearchCV

parameters = {
    'C': [0.1, 1, 10, 100, 1000],
                # 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                'kernel': ['rbf', 'linear', 'poly'],
                # 'degree':[1,2,3,4,5,6,7,8,9,10],
                # 'class_weight':['balanced',None],
                # 'decision_function_shape':['ovo','ovr']
                }
model=SVC()

clf = GridSearchCV(model, parameters,scoring='f1_macro',cv=5)
clf.fit(x, y[:,0])

print(clf.best_params_)
print(clf.best_score_)


model=tree.DecisionTreeClassifier(**clf.best_params_)
classificationResults(x_train,x_test,y_train,y_test,[model])


In [ ]:
# import random parametes search
from sklearn.model_selection import RandomizedSearchCV
#random hyperparameters search for svm 
model= SVC()

param_grid = {'C': [0.1, 1, 10, 100, 1000],
                'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                'kernel': ['rbf', 'linear', 'poly'],
                # 'degree':[1,2,3,4,5,6,7,8,9,10],
                # 'class_weight':['balanced',None],
                # 'decision_function_shape':['ovo','ovr']
                }
random_search = RandomizedSearchCV(model, param_grid, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs = -1)
random_search.fit(x_train, y_train[:,0])
print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.best_estimator_)
# classificationResults(x_train,x_test,y_train,y_test,random_search.best_estimator_)




In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
parameters={
    'n_estimators': [50,70,100,120],
    "max_depth": range(2,20),
    "criterion": ["gini", "entropy"],
    "min_samples_leaf"=range(1,10),
    "min_samples_split": range(1,10),
    "max_features": ["auto", "sqrt", "log2"]
    "class_weight":['balanced',None]
    }
model=RandomForestClassifier()

clf = GridSearchCV(model, parameters)
clf.fit(x_train, y_train[:,0])

print(clf.best_params_)
print(clf.best_score_)


In [ ]:
#test all regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
models=[RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor(),\
LinearRegression(),SVR(),
KNeighborsRegressor(),
DecisionTreeRegressor()]
regressionResults(x_train,x_test,y_train,y_test,models)

In [ ]:
# import random parametes search
from sklearn.model_selection import RandomizedSearchCV
#random hyperparameters search for svm 
model= SVC()

param_grid = {'C': [0.1, 1, 10, 100, 1000],
                'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                'kernel': ['rbf', 'linear', 'poly', 'rbf', 'linear'],
                'degree':[1,2,3,4,5,6,7,8,9,10],
                'class_weight':['balanced',None],
                'decision_function_shape':['ovo','ovr']
                }
random_search = RandomizedSearchCV(model, param_grid, n_iter=10, cv=5, verbose=2, random_state=42, n_jobs = -1)
random_search.fit(x_train, y_train[:,0])
print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.best_estimator_)
# classificationResults(x_train,x_test,y_train,y_test,random_search.best_estimator_)




In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
parameters={
    'n_estimators': [50,70,100,120],
    "max_depth": range(2,20),
    "criterion": ["gini", "entropy"],
    "min_samples_leaf"=range(1,10),
    "min_samples_split": range(1,10),
    "max_features": ["auto", "sqrt", "log2"]
    "class_weight":['balanced',None]
    }
model=RandomForestClassifier()

clf = GridSearchCV(model, parameters)
clf.fit(x_train, y_train[:,0])

print(clf.best_params_)
print(clf.best_score_)


In [ ]:
model=RandomForestClassifier(class_weight=class_weight)
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
model=RandomForestClassifier(class_weight='balanced')
classificationResults(x_train,x_test,y_train,y_test,model)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# from sklearn import svm
# parameters={'kernel':('linear', 'poly' ,"rbf","sigmoid"),'gamma':["scale","auto"],'class_weight':[None,"balanced"]}


# model=svm.SVC()

# clf = GridSearchCV(model, parameters)
# clf.fit(x_train, y_train[:,0])

# print(clf.best_params_)
# print(clf.best_score_)


In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# model=AdaBoostClassifier()
# print(f" those are the parameters to optimize {model.get_params()}")
# parameters={"n_estimators":[10,5,20,50,100,200,500,1000],"learning_rate":[0.01,0.001,0.0001,0.05,0.1,0.2,0.5,1]}
# clf = GridSearchCV(model, parameters)
# clf.fit(x_train, y_train[:,0])

# print(clf.best_params_)
# print(clf.best_score_)


In [ ]:
#test all regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
models=[RandomForestRegressor(),
AdaBoostRegressor(),
# GradientBoostingRegressor(),
LinearRegression(),
SVR(),
BayesianRidge(),
# TweedieRegressor(),
KNeighborsRegressor(),
DecisionTreeRegressor()]
regressionResults(x_train,x_test,y_train,y_test,models)

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
parameters={
    'n_estimators': [10000],
    "max_depth": [100],
}
model=RandomForestRegressor()
clf = GridSearchCV(model, parameters,scoring="neg_mean_squared_error",cv=5)
clf.fit(x_train, y_train[:,1])
print(clf.best_params_)
print(clf.best_score_)
print(clf.best_estimator_)




In [ ]:
regressionResults(x_train,x_test,y_train,y_test,clf.best_estimator_)